In [21]:
from memo import memo
import jax.numpy as np
import jax
from jax.scipy.stats import expon, uniform, norm
from matplotlib import pyplot as plt

Space of possible tournaments

In [56]:
Players = np.arange(1, 5)
MaxRounds = np.arange(1, 5)
Strengths = np.arange(0, 100)
Efforts = [0, 1]
# Enumerate over tournaments
import itertools
all_matches = [list(match) for match in itertools.permutations(Players, 2)] # 1v1 matches

# Generate all tournaments of length 1 to 4 matches
# Lio: note that this is 'redundant' bc there will be Player A, Player A and also Player B, Player B
# All of these tournaments refer to "Team 1 beat Team 2"
MaxLengthTournaments = 2
SinglesTournaments = []
for r in range(1, MaxLengthTournaments + 1):  
    # combinations with replacement allows repeated matches across tournaments
    # using product to allow repeated matches *within* a tournament
    for combo in itertools.product(all_matches, repeat=r):
        SinglesTournaments.append([list(match) for match in combo])

print(len(SinglesTournaments))
print(SinglesTournaments[-1])

156
[[Array(4, dtype=int32), Array(3, dtype=int32)], [Array(4, dtype=int32), Array(3, dtype=int32)]]


Tug-of-war listener in memo

In [31]:
@jax.jit
def pdf_Gaussian(difference, sigma):
    return norm.pdf(difference, scale=sigma) #Gaussian noise

@memo
def tow_listener[player : Players, tournaments: SinglesTournaments](strength_difference=50, strength_sigma=10, lazy_bernoulli=0.3):
    listener: thinks [
        player: assigned (strength in Strengths, wpp=pdf_Gaussian(strength - strength_difference, strength_sigma)),
        player: assigned (effort in Efforts, wpp=lazy_bernoulli if effort else 1 - lazy_bernoulli),
        
    ]
    return listener[E[player.strength]]
tow_listener(print_table=True)

+-----------------+--------------------+
| player: Players | tow_listener       |
+-----------------+--------------------+
| 1               | 49.99999237060547  |
| 2               | 49.99999237060547  |
| 3               | 49.99999237060547  |
+-----------------+--------------------+


Array([49.999992], dtype=float32)

Generate possible narratives